In [1]:
import rasterio
from rasterio.enums import Resampling
from rasterio.warp import reproject
import numpy as np
import os

# Paths to the input GeoTIFF files
raster1_path = 'UK_DEM2.tif'
raster2_path = 'UK_temp2.tif'

# List of resampling methods
resampling_methods = [
    Resampling.nearest,
    Resampling.bilinear,
    Resampling.cubic,
    Resampling.cubic_spline,
    Resampling.lanczos,
    Resampling.average,
    Resampling.mode,
    Resampling.max,
    Resampling.min,
    Resampling.med,
    Resampling.q1,
    Resampling.q3,
    Resampling.sum,
    Resampling.rms
]

# Function to create a binary mask where data is present
def create_presence_mask(data, nodata_value):
    mask = np.ones_like(data, dtype=np.uint8)
    if nodata_value is not None and np.isnan(nodata_value):
        mask[np.isnan(data)] = 0
    elif nodata_value is not None:
        mask[data == nodata_value] = 0
    return mask

# Open the input rasters
with rasterio.open(raster1_path) as src1:
    data1 = src1.read(1)
    meta1 = src1.meta
    nodata1 = src1.nodata

with rasterio.open(raster2_path) as src2:
    data2 = src2.read(1)
    nodata2 = src2.nodata

for method in resampling_methods:
    # Folder to save the output files
    output_folder = method.name.lower()
    os.makedirs(output_folder, exist_ok=True)
    
    # Extract folder name for dynamic naming
    folder_name = os.path.basename(output_folder)
    
    output_path_difference_mask = os.path.join(output_folder, f'UK_imputation_raster_{folder_name}.tif')
    output_path_resampled_mask = os.path.join(output_folder, f'UK_resampled_mask_{folder_name}.tif')
    
    # Resample raster2 to match raster1 if dimensions differ
    if src1.shape != src2.shape:
        data2_resampled = np.empty(src1.shape, dtype=np.float32)
        reproject(
            source=data2,
            destination=data2_resampled,
            src_transform=src2.transform,
            src_crs=src2.crs,
            dst_transform=src1.transform,
            dst_crs=src1.crs,
            resampling=method
        )
    else:
        data2_resampled = data2
    
    # Create presence masks for each raster
    mask1 = create_presence_mask(data1, nodata1)
    mask2 = create_presence_mask(data2_resampled, nodata2)
    
    # Save the resampled mask
    meta_resampled_mask = src1.meta.copy()
    meta_resampled_mask.update(dtype=rasterio.uint8, nodata=0)
    with rasterio.open(output_path_resampled_mask, 'w', **meta_resampled_mask) as dst:
        dst.write(mask2, 1)
    
    # Calculate the difference mask
    difference_mask = mask1 & ~mask2
    
    # Update metadata for the output raster
    meta1.update(dtype=rasterio.uint8, nodata=0)
    
    # Save the difference mask as a new GeoTIFF file
    with rasterio.open(output_path_difference_mask, 'w', **meta1) as dst:
        dst.write(difference_mask, 1)

    print(f"The difference raster and resampled mask have been created and saved using {method.name} resampling.")

The difference raster and resampled mask have been created and saved using nearest resampling.
The difference raster and resampled mask have been created and saved using bilinear resampling.
The difference raster and resampled mask have been created and saved using cubic resampling.
The difference raster and resampled mask have been created and saved using cubic_spline resampling.
The difference raster and resampled mask have been created and saved using lanczos resampling.
The difference raster and resampled mask have been created and saved using average resampling.
The difference raster and resampled mask have been created and saved using mode resampling.
The difference raster and resampled mask have been created and saved using max resampling.
The difference raster and resampled mask have been created and saved using min resampling.
The difference raster and resampled mask have been created and saved using med resampling.
The difference raster and resampled mask have been created an

Collecting File Paths

In [7]:
import os
from rasterio.enums import Resampling

# Define the resampling methods list
resampling_methods = [
    Resampling.nearest,
    Resampling.bilinear,
    Resampling.cubic,
    Resampling.cubic_spline,
    Resampling.lanczos,
    Resampling.average,
    Resampling.mode,
    Resampling.max,
    Resampling.min,
    Resampling.med,
    Resampling.q1,
    Resampling.q3,
    Resampling.sum,
    Resampling.rms
]

# Dictionary to store the file paths
imputation_raster_files = {}

# Add the paths to the dictionary
for method in resampling_methods:
    folder_name = method.name.lower()
    # Construct the directory path with forward slashes and the 'temp/' prefix    NEED TO CHANGE here
    directory_path = os.path.join('Permutations', folder_name).replace('\\', '/')
    # Construct the file path within the directory
    file_path = os.path.join(directory_path, f'UK_imputation_raster_{folder_name}.tif').replace('\\', '/')
    imputation_raster_files[directory_path] = file_path


print(imputation_raster_files)


{'Permutations/nearest': 'Permutations/nearest/UK_imputation_raster_nearest.tif', 'Permutations/bilinear': 'Permutations/bilinear/UK_imputation_raster_bilinear.tif', 'Permutations/cubic': 'Permutations/cubic/UK_imputation_raster_cubic.tif', 'Permutations/cubic_spline': 'Permutations/cubic_spline/UK_imputation_raster_cubic_spline.tif', 'Permutations/lanczos': 'Permutations/lanczos/UK_imputation_raster_lanczos.tif', 'Permutations/average': 'Permutations/average/UK_imputation_raster_average.tif', 'Permutations/mode': 'Permutations/mode/UK_imputation_raster_mode.tif', 'Permutations/max': 'Permutations/max/UK_imputation_raster_max.tif', 'Permutations/min': 'Permutations/min/UK_imputation_raster_min.tif', 'Permutations/med': 'Permutations/med/UK_imputation_raster_med.tif', 'Permutations/q1': 'Permutations/q1/UK_imputation_raster_q1.tif', 'Permutations/q3': 'Permutations/q3/UK_imputation_raster_q3.tif', 'Permutations/sum': 'Permutations/sum/UK_imputation_raster_sum.tif', 'Permutations/rms': '

Make Sure you have the points shape file for the temprature raster file to get  grid points so that you can delete colliding extras.

In [ ]:
import rasterio
from rasterio.enums import Resampling
import numpy as np
import os
from shapely.geometry import Point
import geopandas as gpd
from tqdm import tqdm
import pandas as pd

# Paths to the input GeoTIFF files
raster1_path = 'UK_temp2.tif'  # This is the reference raster for resolution
comparison_shapefile_path = 'UK_temp2.shp'  # Path to the comparison shapefile

# Dictionary of folders and corresponding raster paths
raster_dict = imputation_raster_files

# List of resampling methods
resampling_methods = [
    Resampling.bilinear,
    Resampling.cubic,
    Resampling.cubic_spline,
    Resampling.lanczos,
    Resampling.average,
    Resampling.mode,
    Resampling.max,
    Resampling.min,
    Resampling.med,
    Resampling.q1,
    Resampling.q3,
    Resampling.sum,
    Resampling.rms
]

# Function to create a binary mask where data is present
def create_presence_mask(data, nodata_value):
    mask = np.ones_like(data, dtype=np.uint8)
    if nodata_value is not None and np.isnan(nodata_value):
        mask[np.isnan(data)] = 0
    elif nodata_value is not None:
        mask[data == nodata_value] = 0
    return mask

# Open the reference raster to get its metadata and resolution
with rasterio.open(raster1_path) as src_ref:
    # Read the data from the first band
    data_ref = src_ref.read(1)
    profile_ref = src_ref.profile
    transform_ref = src_ref.transform  # Get the transform for resampling

# Load the comparison shapefile
gdf_comparison = gpd.read_file(comparison_shapefile_path)

# DataFrame to store results
results_df = pd.DataFrame(columns=['shapefile_path', 'imputation_raster', 'number_of_points'])

# Process each folder and its corresponding raster file
for folder, raster2_path in tqdm(raster_dict.items(), desc="Processing folders"):
    # Process each resampling method
    for method in tqdm(resampling_methods, desc="Processing resampling methods", leave=False):
        # Folder to save the output files
        output_folder = os.path.join(folder, method.name.lower())
        os.makedirs(output_folder, exist_ok=True)
        
        output_path = os.path.join(output_folder, f'UK_imputation_resampled_{method.name.lower()}.tif')
        shapefile_path = os.path.join(output_folder, f'UK_imputation_{method.name.lower()}.shp')

        # Open the raster to be resampled
        with rasterio.open(raster2_path) as src:
            # Read the data from the first band
            data = src.read(1)
            profile = src.profile

            # Resample raster2 to match raster1 if dimensions differ
            if src_ref.shape != src.shape:
                resampled_data = np.empty(src_ref.shape, dtype=np.float32)
                rasterio.warp.reproject(
                    source=rasterio.band(src, 1),
                    destination=resampled_data,
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform_ref,
                    dst_crs=profile_ref['crs'],
                    resampling=method
                )
            else:
                resampled_data = data  # No need to resample if already matching

        # Create a presence mask for the resampled raster
        mask_resampled = create_presence_mask(resampled_data, src.nodata)

        # Lists to store point geometries, coordinates, and pixel values
        points = []
        lats = []
        lons = []
        values = []

        # Get the number of rows and columns in the mask
        rows, cols = mask_resampled.shape

        # Iterate through each pixel in the mask
        for row in range(rows):
            for col in range(cols):
                # Read pixel value
                pixel_value = mask_resampled[row, col]

                # Check if pixel value is not nodata and not NaN
                if pixel_value != 0 and not np.isnan(pixel_value):
                    # Convert pixel coordinates to geographic coordinates (longitude, latitude)
                    lon, lat = src_ref.xy(row, col)

                    # Create a point geometry for each valid pixel coordinate
                    point = Point(lon, lat)
                    points.append(point)
                    lats.append(lat)
                    lons.append(lon)
                    values.append(pixel_value)

        # Create a GeoDataFrame from the list of points
        gdf = gpd.GeoDataFrame(geometry=points, crs=src_ref.crs)

        # Add latitude, longitude, and pixel values to the GeoDataFrame
        gdf['latitude'] = lats
        gdf['longitude'] = lons
        gdf['value'] = values

        # Ensure both GeoDataFrames have the same CRS
        if gdf.crs != gdf_comparison.crs:
            gdf_comparison = gdf_comparison.to_crs(gdf.crs)

        # Identify points in gdf that are not in gdf_comparison based on latitude and longitude
        gdf_filtered = gdf[~gdf.apply(lambda x: (x['latitude'], x['longitude']) in 
                                    set(gdf_comparison.apply(lambda y: (y['latitude'], y['longitude']), axis=1)), axis=1)]

        num_points = len(gdf_filtered) if not gdf_filtered.empty else 0

        # Save the results to the DataFrame
        new_row = pd.DataFrame({
            'shapefile_path': [shapefile_path],
            'imputation_raster': [raster2_path],
            'number_of_points': [num_points]
        })
        results_df = pd.concat([results_df, new_row], ignore_index=True)

        # Check if gdf_filtered is not empty before saving
        if not gdf_filtered.empty:
            # Create a GeoDataFrame for the filtered points
            gdf_filtered = gpd.GeoDataFrame(gdf_filtered, geometry=gdf_filtered.geometry, crs=gdf.crs)

            # Save the filtered GeoDataFrame to a new shapefile
            gdf_filtered.to_file(shapefile_path)

        # Update metadata for the output raster
        profile_ref.update(
            dtype=rasterio.uint8,  # Update data type if necessary
            nodata=0  # Define nodata value
        )

        # Save the resampled mask as a new GeoTIFF file
        with rasterio.open(output_path, 'w', **profile_ref) as dst:
            dst.write(mask_resampled, 1)

        # Display progress and status
        tqdm.write(f"Resampled mask using {method.name} resampling has been created and saved in {output_folder}.")
        tqdm.write(f"Filtered shapefile for {method.name} resampling has been created and saved in {output_folder}.")

# Save the results DataFrame to a CSV file
results_df.to_csv('results.csv', index=False)

print("Processing complete. Results have been saved to 'results.csv'.")


Making Data frame out of results CSV file

In [11]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('results.csv')

# Initialize an empty list to store the extracted values
data = []

# Loop through each row in the dataframe
for index, row in df.iterrows():
    path = row['shapefile_path']
    points = row['number_of_points']
    
    # Extract the relevant parts from the path
    parts = path.split('\\')
    if len(parts) > 2:
        method1 = parts[0].split('/')[-1]
        method2 = parts[1]
    else:
        method1 = parts[0].split('/')[-1]
        method2 = parts[0].split('/')[-1]
    
    # Append the extracted data to the list
    data.append([method1, method2, points])

# Convert the list to a dataframe
result_df = pd.DataFrame(data, columns=['Method1', 'Method2', 'Points'])

print(result_df)



     Method1       Method2  Points
0    nearest      bilinear      64
1    nearest         cubic      64
2    nearest  cubic_spline      64
3    nearest       lanczos       0
4    nearest       average     372
..       ...           ...     ...
177      rms           med     377
178      rms            q1     377
179      rms            q3     377
180      rms           sum     380
181      rms           rms     377

[182 rows x 3 columns]


Make a pivot table 